# RFNSA Antenna Details
##### ©Haris Hassan


In [ ]:
# Import Libraries
import sys
import re
from IPython import get_ipython;
get_ipython().magic('reset -sf')
import pandas as pd
pd.set_option('display.max_colwidth', None)
from IPython.display import display, HTML
import re
import math
from itertools import count

In [ ]:
class text_format:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'
    
# Function to convert from W to dBm
def watts_to_dBm(mW):
    return 10.*log10(mW) + 30

# Function to convert from dBm to W
def dBm_to_watts(dBm):
    return 10**((dBm-30)/10)

def find_mount_height(data):   
    MountHeight = []
    for l, m in zip(data['Height - Phase Centre (m)'], data['Mount Height (m)']):
        if m == 0:
            MountHeight.append(l)
        else:
            MountHeight.append(m)
    return MountHeight

def find_carriers(df):
    Carriers = []
    for idx in list( dict.fromkeys(df['ID'],df['Antenna']) ):
        multicarrier = []
        counter = 0
        for x in df.loc[df['ID'] == idx, 'Operator']:
            multicarrier.append(x)
            counter +=1
        for c in range(counter):
            Carriers.append('/'.join(list( dict.fromkeys(multicarrier))))
    return Carriers

def find_tech(df):
    tech = []
    for x, y in enumerate(df['Carrier']):
        if '/' in y:
            tech.append(''.join(df['System'][x].split(";")))
        else:
            tech.append(df['System'][x].split(";")[1].strip())
    return tech

# Find the sectors
def find_sectors(df):
    sectors = []
    for idx in list( dict.fromkeys(df['ID'],df['Antenna'])):
        Sectorstemp = []
        counter = 0
        for x in df.loc[df['ID'] == idx, 'Sector']:
            Sectorstemp.append(str(x))
            counter +=1
            #display(idx)
        Sectorstemp.sort()
        for c in range(counter):
            sectors.append(', '.join(list( dict.fromkeys(Sectorstemp))))
    return sectors

######## what should be the Assessment frequency? ########
def Find_assesment_frequency(TECH):
    Assess_Freq_list = []
    for t in TECH:
        try:
            i = re.search("(\d+)", str(t)).group(0)
            if any([x == i for x in ['900','850']]):
                Assess_Freq_list.append('900')
            elif any([i == 2100]):
                Assess_Freq_list.append('2100')
            elif i == '1800':
                Assess_Freq_list.append('1800')
            elif i== '2600':
                Assess_Freq_list.append('2600')
            elif any([x == i for x in ['700','750']]):
                Assess_Freq_list.append('750')
            elif i == '2300':
                Assess_Freq_list.append('2350')
            elif i == '3500':
                Assess_Freq_list.append('3500')
            else:
                Assess_Freq_list.append(i)
        except AttributeError:
            Assess_Freq_list.append(t.strip())
    return Assess_Freq_list

#Correcting microwavelinks
def fix_microwavelinks_power(df):
    templist = []
    for x in df['Powers']:
        if ';' not in str(x):
            templist.append(str(x)+';')
        else:
            templist.append(str(x))
    return templist

#Count the total ports of each antenna
def count_total_ports(df):
    total_ports = df['Powers'].str.count(';').fillna(0) + 1
    return total_ports

### Convert te Power(dBm) into Watts
def find_power_in_watts(df):
    temp_power = []
    for elm in df['Powers'].str.split(";"):
        if isinstance(elm, (list, tuple)):
            elm = [x for x in elm if x != '']  # Remove any empty strings from the list
            converted_values = []
            for sub in elm:
                if float(sub) >= 30:
                    try:
                        converted_values.append(round(dBm_to_watts(float(sub)),1))
                    except ValueError:
                        converted_values.append(sub)
                        pass  # Skip the current value if it cannot be converted to a float
                else:
                     try:
                        converted_values.append(round(dBm_to_watts(float(sub)),4))
                     except ValueError:
                        converted_values.append(sub)
                        pass  # Skip the current value if it cannot be converted to a float
            temp_power.append(converted_values)
        else:
            try:
                temp_power.append([dBm_to_watts(float(elm))])
            except ValueError:
                temp_power.append([])
    return temp_power

######## Which ports to add powers to? ########
def find_powered_ports(df):
    Mylist =[]

    for i in df['Powers']:
        count = 1
        lister = []
        if isinstance(i, str):
            for j in i:
                if j==';':
                    count+=1
                else:
                    lister.append(count)
            Mylist.append(lister)
        else:
            Mylist.append(list(str(i))) 

    new_k = []
    for elem in Mylist:
        kiterator=[]
        for el in elem:
            if el not in kiterator:
                kiterator.append(el)
        new_k.append(kiterator)
    k = new_k
    return k

#Add 0 before single digit
def fix_alphanumeric(input_list):
    fixed_list = []
    for x in input_list:
        templist = []
        for y in re.split(r'(\d+)', x.strip()):
            if re.match('\d{1,2}', y):
                y = y.zfill(3)
            templist.append(y)
        fixed_list.append(''.join(templist))
    return fixed_list

In [ ]:
################### Replace with path to your excel file. 
data = pd.read_excel(r'C:\Users\Mewtwo\Downloads\New Microsoft Excel Worksheet.xlsx')
data.fillna(value = 0,inplace = True)
#remove the proposed Antennas
data = data.drop(data[data['Existing/Proposed'] == 'Proposed'].index)
data['Height'] = find_mount_height(data)

In [ ]:
#dfSiteInfo = pd.DataFrame(data, columns=['Structure Owner','Latitude', 'Longitude', 'Region', 'Structure',
#                                         'Site System (Structure No)','NSA','Site Name'])
dfSiteInfo = pd.DataFrame(data, columns=['NSA','Latitude', 'Longitude', 'Region'])
dfSiteInfo['Coordinates'] = dfSiteInfo.apply(lambda row: str(row['Latitude']) + ', ' + str(row['Longitude']), axis=1)
dfSiteInfo = dfSiteInfo.drop('Latitude', axis=1)
dfSiteInfo = dfSiteInfo.drop('Longitude', axis=1)

dfStructureInfo = pd.DataFrame(data, columns=['Structure Owner','Structure',
                                         'Site System (Structure No)',])
dfSiteNames = pd.DataFrame(data, columns=['System','Site Name'])
dfSiteNames['Operator'] = dfSiteNames['System'].str.split(" /").str[0]
dfSiteNames = dfSiteNames.drop('System', axis=1)
dfSiteNames['Site Name']=dfSiteNames['Site Name'].apply(lambda x: x.title())

dfStructureInfo['Structure'] = ['Structure ' +f'{l}' + ': ' + ''.join(''.join(map(str, m))) for l, m in zip(dfStructureInfo['Site System (Structure No)'],dfStructureInfo['Structure'])]
dfStructureInfo = dfStructureInfo.drop('Site System (Structure No)', axis=1)


dfSiteNames = dfSiteNames.drop_duplicates(keep='first')
dfSiteInfo = dfSiteInfo.drop_duplicates(keep='first')
dfStructureInfo = dfStructureInfo.drop_duplicates(keep='first')

In [ ]:
################ Create the dataframe from meaningful columns of STAD table and rename the columns ################
df = pd.DataFrame(data, columns=['Antenna ID No','Add ID','Antenna Model','Sector','Height'
                                 ,'Bearing Degrees (true)',
                                 'Mech Downtilt','Elect Downtilt','System',
                                 'Port Number (Band Power per Port (dBm))','Band Power per Port (dBm)','Notes'])
df = df.rename(columns={'Antenna Model': 'Antenna', 'Antenna ID No': 'ID', 'Bearing Degrees (true)': 'Bearing'
                  , 'Mech Downtilt': 'MDT', 'Elect Downtilt': 'EDT', 
                   'Port Number (Band Power per Port (dBm))': 'Possible Ports', 
                   'Band Power per Port (dBm)': 'Powers'})
df.fillna(value = 0,inplace = True)
df = df.reset_index(drop=True)


In [ ]:
# Extracting Carrier (optus, telstra, Vodafone), technology/Frequency (i.e LTE900, NR2100, WCDMA850) information from System 
df['System'] = df['System'].str.replace(' - LOCKED', '', regex=False)
df['System'] = df['System'].str.replace('[Macro]', '', regex=False)
df['System'] = df['System'].str.replace('/', ';', 1, regex=False)
df['Operator'] = df['System'].str.split(" ;").str[0]
df['Carrier'] = find_carriers(df)
df['Tech'] = find_tech(df)

In [ ]:
df["sorter"] =  fix_alphanumeric(df["Carrier"] + df["ID"].astype(str)) 
df.sort_values(by=['sorter'], inplace=True)
df = df.reset_index(drop=True)
df.index += 1

df['Total Ports'] = count_total_ports(df)
df['EDT'] = df['EDT'].str.replace(' to ', '-', regex=False).replace(r'\(.*\)','', regex=True)
df['Sector'] = df['Sector'].astype(str).replace('\.0', '', regex=True)
df['System/Sector'] = [''.join(''.join(map(str, m))+' - Sector ' +f'{l}') for l, m in zip(df['Sector'],df['Tech'])]
df['Powers'] = fix_microwavelinks_power(df)
df['Sectors'] = find_sectors(df)
df['Powers (W)'] = find_power_in_watts(df)
df['Assess Freq'] = Find_assesment_frequency(df['Tech'])
df['WhereToAddPower'] = find_powered_ports(df)

## Add these Antennas to prox5

In [ ]:
AntennasUnique = list( dict.fromkeys(df['Antenna']) )
IDUnique = list( dict.fromkeys(df['ID'],df['Antenna']) )
AntennaIds = {}
for antenna in AntennasUnique:
    IdsForAntenna = list(dict.fromkeys(df.loc[df['Antenna'] == antenna, 'ID']))
    AntennaIds[antenna] = IdsForAntenna 
#print('You need to add these antennas\n\n'+str.join(" \n", AntennasUnique)+' \n\nto PROX5')

## IDs for Antennas

In [ ]:
for x, y in AntennaIds.items():
    #print(f'\nThe id of '+text_format.BOLD+ x + text_format.END + ' are\n'+str.join("", str(y)))


## Antenna Settings

In [ ]:
display('Set Power and Frequency to ports in Prox5')
for i in IDUnique:
        #print(f'\nFor '+text_format.BOLD+str(i)+text_format.END+f', the Settings are')
        #display(df.loc[(df['ID'] == i),['ID','Height','Bearing','MDT']])

## Set Power and Frequency to ports in Prox5

In [ ]:
display('Set Power and Frequency to ports in Prox5')
for i in IDUnique:
        #print(text_format.BOLD+str(i))
        #display(df.loc[(df['ID'] == i),['ID','EDT','Tech','Possible Ports','WhereToAddPower','Powers (W)']])

## EMEG Equipment List

In [ ]:
totalPortsdict = {}
for idantenna in list( dict.fromkeys(df['ID'],df['Antenna'])):
    totalPorts = list(dict.fromkeys(df.loc[df['ID'] == idantenna, 'Total Ports']))
    totalPortsdict[idantenna] = totalPorts
    
listpow = []
listsec = []
x = 0
yprev = 0
for AntId, AntPorts in totalPortsdict.items():
    templistpow = [''] * (int(AntPorts[0] / 2) + int(AntPorts[0] % 2))
    templistsec = [''] * (int(AntPorts[0] / 2) + int(AntPorts[0] % 2))
    tempdf = df.loc[(df['ID']==AntId), ['EDT','System/Sector','WhereToAddPower','Powers (W)']]
    for sector, port, power in zip(tempdf['System/Sector'], tempdf['WhereToAddPower'], tempdf['Powers (W)']):
        display(AntId)
        if len(port)>1:
            for x, y in enumerate(port):       
                if sector not in templistsec[math.ceil(int(y)/2)-1]:
                    templistsec[math.ceil(int(y)/2)-1] += sector +'\n'
                    templistpow[math.ceil(int(y)/2)-1] += '\n'
                templistpow[math.ceil(int(y)/2)-1] += '+' + str(power[x])
        if len(port)==1:
            for x, y in enumerate(port):
                display(math.ceil(int(y)/2)-1)
                templistsec[math.ceil(int(y)/2)-1] += sector +'\n'
                if y%2==0:
                    templistpow[math.ceil(int(y)/2)-1] += '\n0++' + str(power[x])
                else:
                    templistpow[math.ceil(int(y)/2)-1] += '\n' + str(power[x]) + '++0'
            
    listpow.append(templistpow)
    listsec.append(templistsec)

for index1, x in enumerate(listpow):
    for index2, y in enumerate(x):
        if y == '':
            listpow[index1][index2] = '0'
        
for index1, x in enumerate(listsec):
    for index2, y in enumerate(x):
        if y == '':
            listsec[index1][index2] = '-'
        

EquipmentList = pd.DataFrame(totalPortsdict.items(), columns=['ID', 'No. of ports'])
EquipmentList['System/Sector'] = listsec
EquipmentList['Power (W)'] = listpow

EquipmentList_final = pd.DataFrame([], columns=['ID','System/Sector', 'Power (W)'])

for a, x, y in zip(EquipmentList['ID'],EquipmentList['System/Sector'], EquipmentList['Power (W)']): 
        EquipmentListAntenna = pd.DataFrame([a]*len(x),columns=['ID'])
        EquipmentListAntenna['System/Sector'] = x
        EquipmentListAntenna['Power (W)'] = y
        EquipmentList_final = pd.concat([EquipmentList_final, EquipmentListAntenna], axis=0)

EquipmentList_final['Power (W)'] = EquipmentList_final['Power (W)'].str.replace('\+', '', 1, regex=True)
EquipmentList_final['Power (W)'] = EquipmentList_final['Power (W)'].str.replace('\+\+', '+', regex=True)
EquipmentList_final['Power (W)'] = EquipmentList_final['Power (W)'].str.replace('\\n\+', '\\n', regex=True)
EquipmentList_final['Power (W)'] = EquipmentList_final['Power (W)'].str.replace('+', ' + ', regex=False)
EquipmentList_final['Power (W)'] = EquipmentList_final['Power (W)'].str.strip()

EquipmentList_final['System/Sector'] = EquipmentList_final['System/Sector'].str.replace('\n ', '\n', regex=False)
EquipmentList_final['System/Sector'] = EquipmentList_final['System/Sector'].str.strip()
#display(EquipmentList_final)


## CANRAD

In [ ]:
listpow = []
listfunc = []
listport = []
x = 0
yprev = 0

for AntId, AntPorts in totalPortsdict.items():
    templistpow = [0] * int(AntPorts[0])
    templistfunc = [''] * int(AntPorts[0])
    templistport = list(range(1,int(AntPorts[0])+1))
    #tempfunc = [''] * int(AntPorts[0])
    tempdf = df.loc[(df['ID']==AntId), ['Sector','Tech','WhereToAddPower','Powers (W)']]
    for sector, tech, port, power in zip(tempdf['Sector'], tempdf['Tech'], tempdf['WhereToAddPower'], tempdf['Powers (W)']):
        for x, y in enumerate(port):       
            #if sector not in templistfunc[y]:
            templistfunc[y-1] += tech +'/'
            templistpow[y-1] = str(round(float(templistpow[y-1]) + float(power[x]),3))
    listpow.append(templistpow)
    listport.append(templistport)
    listfunc.append(templistfunc)

CANRAD = pd.DataFrame(totalPortsdict.items(), columns=['ID', 'No. of ports'])
CANRAD['Port'] = listport
CANRAD['Function'] = listfunc
CANRAD['Total Power'] = listpow

CANRAD_final = pd.DataFrame([], columns=['ID','Function', 'Total Power'])

for a, z, x, y in zip(CANRAD['ID'], CANRAD['Port'],CANRAD['Function'], CANRAD['Total Power']): 
        CANRADAntenna = pd.DataFrame([a]*len(x),columns=['ID'])
        CANRADAntenna['Port'] = z
        CANRADAntenna['Function'] = x
        CANRADAntenna['Total Power'] = y
        CANRAD_final = pd.concat([CANRAD_final, CANRADAntenna], axis=0)
CANRAD_final=CANRAD_final.iloc[:, [0,3,1,2]]
display(CANRAD_final)


## Exports

In [ ]:
#df3 = pd.DataFrame.from_dict(AntennaIds)
df3 = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in AntennaIds.items() ]))
df3 = df3.fillna('')

dfAntennaSettings = df[['Carrier','ID','Height','Bearing','MDT','Total Ports','Sectors']].copy()
dfAntennaSettings = dfAntennaSettings.drop_duplicates(keep='first')

path = r'C:\Users\Mewtwo\Desktop'+'\\'+ str(data['NSA'][3])+ ' ' + re.sub("[\\\\/]", " ", data['Site Name'][3]) + ' RFNSA Wrangled Data.xlsx'

from openpyxl.utils import get_column_letter
from openpyxl import Workbook
from openpyxl import load_workbook

wb = Workbook()
wb.save(path)

writer = pd.ExcelWriter(path, engine='openpyxl')
df3.to_excel(writer, sheet_name = 'Antenna IDs', index=False)
dfAntennaSettings.to_excel(writer, sheet_name = 'Antenna Settings', index=False)
df.to_excel(writer, sheet_name = 'Add powers to Prox', columns = ['Antenna','ID','EDT','Tech','WhereToAddPower','Powers (W)','Assess Freq','Notes'], index=False)
EquipmentList_final.to_excel(writer, sheet_name = 'EMEG List', index=False)
CANRAD_final.to_excel(writer, sheet_name = 'CANRAD', index=False)
dfSiteInfo.to_excel(writer, sheet_name = 'Site Info', index=False)
dfSiteNames.to_excel(writer, sheet_name = 'Site Info', startcol = 3,index=False)
dfStructureInfo.to_excel(writer, sheet_name = 'Site Info',startcol = 5, index=False)


# load the Excel file with openpyxl
workbook = writer.book

for sheet in workbook:
    for column in range(1, sheet.max_column + 2):
        letter = get_column_letter(column)
        #sheet.column_dimensions[letter].auto_size = True
        sheet.column_dimensions[letter].bestFit = True

# save the modified workbook
workbook.save(path)

In [ ]:

for sheetname in workbook.sheetnames:
    worksheet = workbook[sheetname]

    # set best fit attribute to true for all columns
    for column_cells in worksheet.columns:
        length = max(len(str(cell.value)) for cell in column_cells)
        worksheet.column_dimensions[column_cells[0].column_letter].width = length + 5
        worksheet.column_dimensions[column_cells[0].column_letter].bestFit = True

worksheet = workbook.active

# set the width of column A to 15
column_letter = get_column_letter(1)  # A
column_dimension = worksheet.column_dimensions[column_letter]
column_dimension.width = 15
column_dimension.bestFit = True
